In [9]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import os, sys, inspect

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
from PIL import Image
from io import BytesIO, StringIO
import requests

import dateparser as dp
import datetime

import re

import hashlib

commons_site = pb.Site("commons", "commons")

In [10]:
def is_commons_file (sha):
    commons = ''
    pages = pb.site.APISite.allimages(commons_site, sha1=sha)
    result = False
    for page in pages :
        result = True
        break
    return result

def get_hash (image_file):
    hasher = hashlib.sha1()
    with open(image_file, 'rb') as afile:
        buf = afile.read()
        hasher.update(buf)
    return hasher.hexdigest()

In [11]:
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

req_proxy = RequestProxy()

2017-12-30 23:26:52,605 root   DEBUG    === Initialized Proxy Parsers ===
2017-12-30 23:26:52,607 root   DEBUG    	 FreeProxy Parser of 'http://free-proxy-list.net' with required bandwidth: '5' KBs
2017-12-30 23:26:52,608 root   DEBUG    	 ProxyForEU Parser of 'http://proxyfor.eu/geo.php' with required bandwidth: '1.0' KBs
2017-12-30 23:26:52,609 root   DEBUG    	 RebroWeebly Parser of 'http://rebro.weebly.com' with required bandwidth: '5' KBs
2017-12-30 23:26:52,613 root   DEBUG    	 SemairProxy Parser of 'https://premproxy.com/list/' with required bandwidth: '5' KBs
2017-12-30 23:26:52,614 root   DEBUG    =================================
2017-12-30 23:26:53,184 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x7f816126f208>
2017-12-30 23:26:53,456 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2017-12-30 23:26:53,458 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    P

In [12]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Fototeca: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

In [15]:
urls = ['http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=089b357e17',
'http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=087ddd4818',
'http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=085b9ef61b',
'http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=0881a55b1b',
'http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=08488f1637']

In [ ]:
for url in urls:
    categories = []
    cat_string = ''
    while True:
        request = req_proxy.generate_proxied_request(url)
        if request is not None:
            #print("{0}".format(u''.join(request.text).encode('utf-8')))
            response = request.text
            try :
                soup = BeautifulSoup(response, 'html.parser')
                title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
                break
            except :
                pass
        else :
            print ('Trying')        
    try:
        date_string = soup.find_all("div", class_="contenidoFecha")[0].get_text()
        pub_date_sp = ' '.join(date_string.strip().split(' ')[-4:])
        parsed_pub_date = dp.parse(pub_date_sp)
        pub_date = parsed_pub_date.strftime('%Y-%m-%d')
        year_date = parsed_pub_date.strftime('%Y')
        if 'La Moncloa' in date_string:
            categories.append('Palacio de la Moncloa')
            categories.append(year_date + ' in Madrid')
            cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])
    except :
        pub_date = None
    contents = soup.find_all("div", class_="entradillaContenedor")[0].get_text().strip().replace('\t', '').split('\r\n')
    contents = [item.replace('\n', '') for item in contents if len(item)>0]
    author = contents[0]
    description = contents[1]
    image_url = 'http://www.lamoncloa.gob.es' + soup.find_all("a", id="ctl00_PlaceHolderMain_DisplayMode_aDescargar")[0]['href']
    
    image_name = "%s.jpg" %(title)
    image_path = os.path.join(os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0])), image_name)
    print (image_path)
    while True:
        print ('Trying download')
        img_request = req_proxy.generate_proxied_request(image_url)
        if img_request is not None:
            try :
                bytes = BytesIO(img_request.content)
                image = Image.open(bytes)
                image.save(image_path)
                print ('Downloaded')
                break
            except :
                pass
        else :
            print ('Still trying to download')
    vars = {
        "url": url,
        "description": description,
        "author": author,
        "date": pub_date,
        "title": title,
        "cat_string": cat_string
    }
    t = Template(template)
    _text = t.render(**vars)
    print(_text)

    sha = get_hash(image_path)
    if not is_commons_file(sha) :
        page = pb.Page(commons_site, "File:"+image_name)
        if page.exists():
            print ("Image found in commons. Changing name")
            image_name = image_name.replace('.jpg', '(' + year_date + ').jpg')
        bot = UploadRobot(image_path,
                        description = _text,
                        useFilename = image_name,
                        keepFilename = True,
                        verifyDescription = False,
                        ignoreWarning = True,
                        targetSite = commons_site)
        bot.run()
    else :
        print ("Already in commons. Skipping")
    os.remove(image_path)

2017-12-30 23:43:34,879 root   DEBUG    Using proxy: 45.114.233.149:8080 | FreeProxy
2017-12-30 23:43:42,339 root   DEBUG    Using proxy: 185.2.154.85:4444 | Samair


RR Status 200
/home/paws/José María Aznar preside la Comisión Interministerial de Ciencia y Tecnología.jpg
Trying download
RR Status 200


2017-12-30 23:43:44,945 pywiki VERBOSE  Found 1 commons:commons processes running, including this one.
2017-12-30 23:43:44,951 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2017-12-30 23:43:44,955 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: True
2017-12-30 23:43:44,957 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '103790798524023544111514677424'), ('oauth_timestamp', '1514677424'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07'), ('gaifro

Downloaded
=={{int:filedesc}}==
{{Information
|description={{es|1=El presidente del Gobierno, José María Aznar, preside la Comisión Interministerial de Ciencia y Tecnología, acompañado por la ministra de Ciencia y Tecnología, Anna María Birulés y de la ministra de Educación, Cultura y Deporte, Pilar del Castillo.}}
|date=2002-01-30
|source=[http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=089b357e17 Fototeca: José María Aznar preside la Comisión Interministerial de Ciencia y Tecnología]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa/José María Cuadrado)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Moncloa]]
[[Category:2002 in Madrid]]


2017-12-30 23:43:45,140 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2017-12-30 23:43:45,143 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: True
2017-12-30 23:43:45,145 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '29802847605032149481514677425'), ('oauth_timestamp', '1514677425'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07'), ('titles', 'File:José María Aznar preside la Comisión Interministerial de Ciencia y Tecnología.jpg'), ('inprop',

...

RR Status 200
/home/paws/Pio Cabanillas en la rueda de prensa posterior al Consejo de Ministros junto a la ministra de Ciencia y Tecnología.jpg
Trying download


2017-12-30 23:44:51,501 root   DEBUG    Read timed out - Removed Straggling proxy: 189.10.5.30:3128 | Samair PL Size = 1418
2017-12-30 23:44:51,505 root   DEBUG    Using proxy: 46.35.140.247:8080 | Samair


Still trying to download
Trying download


2017-12-30 23:45:21,544 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 46.35.140.247:8080 | Samair PL Size = 1417
2017-12-30 23:45:21,550 root   DEBUG    Using proxy: 186.188.210.114:53281 | Samair


Still trying to download
Trying download
